<a href="https://colab.research.google.com/github/Aswin-Kumar66/Pyspark_Practise/blob/main/Pyspark_Practise_2___Grouping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=65c5fcca19aec84490e81bcf035c5be19278b6709d46a3881b28416496f5412d
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [6]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.getOrCreate()

## Question 3

In [14]:
df = spark.read.csv('drinks.csv' , header = True , inferSchema = True)

In [15]:
df.show()

+-----------------+-------------+---------------+-------------+----------------------------+---------+
|          country|beer_servings|spirit_servings|wine_servings|total_litres_of_pure_alcohol|continent|
+-----------------+-------------+---------------+-------------+----------------------------+---------+
|      Afghanistan|            0|              0|            0|                         0.0|       AS|
|          Albania|           89|            132|           54|                         4.9|       EU|
|          Algeria|           25|              0|           14|                         0.7|       AF|
|          Andorra|          245|            138|          312|                        12.4|       EU|
|           Angola|          217|             57|           45|                         5.9|       AF|
|Antigua & Barbuda|          102|            128|           45|                         4.9|       NA|
|        Argentina|          193|             25|          221|          

In [17]:
df.printSchema()

root
 |-- country: string (nullable = true)
 |-- beer_servings: integer (nullable = true)
 |-- spirit_servings: integer (nullable = true)
 |-- wine_servings: integer (nullable = true)
 |-- total_litres_of_pure_alcohol: double (nullable = true)
 |-- continent: string (nullable = true)



In [28]:
df.groupby('continent').mean().sort('avg(beer_servings)' , ascending = False).select(['continent' , 'avg(beer_servings)']).show()

+---------+------------------+
|continent|avg(beer_servings)|
+---------+------------------+
|       EU|193.77777777777777|
|       SA|175.08333333333334|
|       NA|145.43478260869566|
|       OC|           89.6875|
|       AF|61.471698113207545|
|       AS| 37.04545454545455|
+---------+------------------+



In [36]:
df.groupby('continent').agg(F.mean('wine_servings') , F.count('wine_servings') , F.std('wine_servings'), F.min('wine_servings') , F.max('wine_servings')).show()

+---------+------------------+--------------------+------------------+------------------+------------------+
|continent|avg(wine_servings)|count(wine_servings)|std(wine_servings)|min(wine_servings)|max(wine_servings)|
+---------+------------------+--------------------+------------------+------------------+------------------+
|       NA| 24.52173913043478|                  23|28.266378301658847|                 1|               100|
|       SA|62.416666666666664|                  12| 88.62018888937148|                 1|               221|
|       AS| 9.068181818181818|                  44|21.667033931944484|                 0|               123|
|       OC|            35.625|                  16| 64.55578982554547|                 0|               212|
|       EU|142.22222222222223|                  45| 97.42173756146497|                 0|               370|
|       AF|16.264150943396228|                  53| 38.84641897335842|                 0|               233|
+---------+--------

In [38]:
df.groupby('continent').mean().show()

+---------+------------------+--------------------+------------------+---------------------------------+
|continent|avg(beer_servings)|avg(spirit_servings)|avg(wine_servings)|avg(total_litres_of_pure_alcohol)|
+---------+------------------+--------------------+------------------+---------------------------------+
|       NA|145.43478260869566|   165.7391304347826| 24.52173913043478|                5.995652173913044|
|       SA|175.08333333333334|              114.75|62.416666666666664|                6.308333333333334|
|       AS| 37.04545454545455|   60.84090909090909| 9.068181818181818|               2.1704545454545454|
|       OC|           89.6875|             58.4375|            35.625|               3.3812500000000005|
|       EU|193.77777777777777|  132.55555555555554|142.22222222222223|                8.617777777777777|
|       AF|61.471698113207545|  16.339622641509433|16.264150943396228|                 3.00754716981132|
+---------+------------------+--------------------+----

In [47]:
 df.createOrReplaceTempView('df1')
spark.sql('select continent, percentile_approx(beer_servings , 0.5),percentile_approx(spirit_servings , 0.5),percentile_approx(wine_servings , 0.5),percentile_approx(total_litres_of_pure_alcohol , 0.5) from df1 group by continent').show()

+---------+--------------------------------------------+----------------------------------------------+--------------------------------------------+-----------------------------------------------------------+
|continent|percentile_approx(beer_servings, 0.5, 10000)|percentile_approx(spirit_servings, 0.5, 10000)|percentile_approx(wine_servings, 0.5, 10000)|percentile_approx(total_litres_of_pure_alcohol, 0.5, 10000)|
+---------+--------------------------------------------+----------------------------------------------+--------------------------------------------+-----------------------------------------------------------+
|       NA|                                         143|                                           137|                                          11|                                                        6.3|
|       SA|                                         162|                                           100|                                           8|                

In [51]:
df.groupby('continent').agg(F.mean('spirit_servings') , F.min('spirit_servings'),F.max('spirit_servings')).show()

+---------+--------------------+--------------------+--------------------+
|continent|avg(spirit_servings)|min(spirit_servings)|max(spirit_servings)|
+---------+--------------------+--------------------+--------------------+
|       NA|   165.7391304347826|                  68|                 438|
|       SA|              114.75|                  25|                 302|
|       AS|   60.84090909090909|                   0|                 326|
|       OC|             58.4375|                   0|                 254|
|       EU|  132.55555555555554|                   0|                 373|
|       AF|  16.339622641509433|                   0|                 152|
+---------+--------------------+--------------------+--------------------+



## Question 4

In [52]:
users = spark.read.csv('u.csv' , header = True , inferSchema = True)

In [53]:
users.show()

+-------+---+------+-------------+--------+
|user_id|age|gender|   occupation|zip_code|
+-------+---+------+-------------+--------+
|      1| 24|     M|   technician|   85711|
|      2| 53|     F|        other|   94043|
|      3| 23|     M|       writer|   32067|
|      4| 24|     M|   technician|   43537|
|      5| 33|     F|        other|   15213|
|      6| 42|     M|    executive|   98101|
|      7| 57|     M|administrator|   91344|
|      8| 36|     M|administrator|    5201|
|      9| 29|     M|      student|    1002|
|     10| 53|     M|       lawyer|   90703|
|     11| 39|     F|        other|   30329|
|     12| 28|     F|        other|    6405|
|     13| 47|     M|     educator|   29206|
|     14| 45|     M|    scientist|   55106|
|     15| 49|     F|     educator|   97301|
|     16| 21|     M|entertainment|   10309|
|     17| 30|     M|   programmer|    6355|
|     18| 35|     F|        other|   37212|
|     19| 40|     M|    librarian|    2138|
|     20| 42|     F|    homemake

In [57]:
users.groupby('occupation').mean().sort('occupation').select(['occupation' , 'avg(age)']).show()

+-------------+------------------+
|   occupation|          avg(age)|
+-------------+------------------+
|administrator| 38.74683544303797|
|       artist|31.392857142857142|
|       doctor| 43.57142857142857|
|     educator| 42.01052631578948|
|     engineer| 36.38805970149254|
|entertainment| 29.22222222222222|
|    executive|          38.71875|
|   healthcare|           41.5625|
|    homemaker| 32.57142857142857|
|       lawyer|             36.75|
|    librarian|              40.0|
|    marketing| 37.61538461538461|
|         none|26.555555555555557|
|        other|34.523809523809526|
|   programmer|33.121212121212125|
|      retired| 63.07142857142857|
|     salesman|35.666666666666664|
|    scientist| 35.54838709677419|
|      student|22.081632653061224|
|   technician|33.148148148148145|
+-------------+------------------+
only showing top 20 rows



In [94]:
users2 = users.groupby('occupation').count()
users2 = users2.withColumnRenamed('count' , 'count1')
users1 = users.filter(F.col('gender') == 'M')
users3 = users1.groupby('occupation').count()
joined = users3.join(users2 , on = 'occupation' )

joined = joined.withColumn('ratio' , joined['count']/joined['count1']).sort('ratio' ,ascending =False)
joined.show()

+-------------+-----+------+-------------------+
|   occupation|count|count1|              ratio|
+-------------+-----+------+-------------------+
|       doctor|    7|     7|                1.0|
|     engineer|   65|    67| 0.9701492537313433|
|   technician|   26|    27| 0.9629629629629629|
|      retired|   13|    14| 0.9285714285714286|
|   programmer|   60|    66| 0.9090909090909091|
|    executive|   29|    32|            0.90625|
|    scientist|   28|    31| 0.9032258064516129|
|entertainment|   16|    18| 0.8888888888888888|
|       lawyer|   10|    12| 0.8333333333333334|
|     salesman|    9|    12|               0.75|
|     educator|   69|    95| 0.7263157894736842|
|      student|  136|   196| 0.6938775510204082|
|        other|   69|   105| 0.6571428571428571|
|    marketing|   16|    26| 0.6153846153846154|
|       writer|   26|    45| 0.5777777777777777|
|         none|    5|     9| 0.5555555555555556|
|administrator|   43|    79| 0.5443037974683544|
|       artist|   15

In [98]:
users.groupby('occupation').agg(F.min('age') , F.max('age')).sort('occupation' ).show()

+-------------+--------+--------+
|   occupation|min(age)|max(age)|
+-------------+--------+--------+
|administrator|      21|      70|
|       artist|      19|      48|
|       doctor|      28|      64|
|     educator|      23|      63|
|     engineer|      22|      70|
|entertainment|      15|      50|
|    executive|      22|      69|
|   healthcare|      22|      62|
|    homemaker|      20|      50|
|       lawyer|      21|      53|
|    librarian|      23|      69|
|    marketing|      24|      55|
|         none|      11|      55|
|        other|      13|      64|
|   programmer|      20|      63|
|      retired|      51|      73|
|     salesman|      18|      66|
|    scientist|      23|      55|
|      student|       7|      42|
|   technician|      21|      55|
+-------------+--------+--------+
only showing top 20 rows



In [107]:
# users_male = users.filter(F.col('gender') == 'M')
# users_female = users.filter(F.col('gender') == 'F')

# join1 = users_male.groupby('occupation').agg(F.mean('age'))
# join2 = users_female.groupby('occupation').agg(F.mean('age'))

# total = join1.join(join2 , on = 'occupation')
# total.sort('occupation').show()

users.groupby(['occupation' , 'gender']).mean().sort('occupation').select(['occupation','gender', 'avg(age)']).show()

+-------------+------+------------------+
|   occupation|gender|          avg(age)|
+-------------+------+------------------+
|administrator|     M| 37.16279069767442|
|administrator|     F|40.638888888888886|
|       artist|     F|30.307692307692307|
|       artist|     M|32.333333333333336|
|       doctor|     M| 43.57142857142857|
|     educator|     F| 39.11538461538461|
|     educator|     M| 43.10144927536232|
|     engineer|     F|              29.5|
|     engineer|     M|              36.6|
|entertainment|     F|              31.0|
|entertainment|     M|              29.0|
|    executive|     M|38.172413793103445|
|    executive|     F|              44.0|
|   healthcare|     F| 39.81818181818182|
|   healthcare|     M|              45.4|
|    homemaker|     F|34.166666666666664|
|    homemaker|     M|              23.0|
|       lawyer|     F|              39.5|
|       lawyer|     M|              36.2|
|    librarian|     M|              40.0|
+-------------+------+------------

In [116]:
gender_count = users.groupby(['occupation' , 'gender']).count()
total_count = users.groupby(['occupation']).count()
total_count = total_count.withColumnRenamed('count' , 'total_count')
final = gender_count.join(total_count , on = 'occupation')
final = final.withColumn('ratio', F.round(final['count']*100/final['total_count'] , 2))
final.sort(['occupation' , 'gender']).select(['occupation' ,'gender', 'ratio']).show()

+-------------+------+-----+
|   occupation|gender|ratio|
+-------------+------+-----+
|administrator|     F|45.57|
|administrator|     M|54.43|
|       artist|     F|46.43|
|       artist|     M|53.57|
|       doctor|     M|100.0|
|     educator|     F|27.37|
|     educator|     M|72.63|
|     engineer|     F| 2.99|
|     engineer|     M|97.01|
|entertainment|     F|11.11|
|entertainment|     M|88.89|
|    executive|     F| 9.38|
|    executive|     M|90.63|
|   healthcare|     F|68.75|
|   healthcare|     M|31.25|
|    homemaker|     F|85.71|
|    homemaker|     M|14.29|
|       lawyer|     F|16.67|
|       lawyer|     M|83.33|
|    librarian|     F|56.86|
+-------------+------+-----+
only showing top 20 rows

